# <p align="center">**5.2 Tiamat: Implementação do Algoritmo Genético** 🧬</p>
 
##### <p align="center">**Autores:**  Glauber Nascimento, Júlia Guedes & Lorena Ribeiro</p>
##### <p align="center">**Orientador:**  Daniel Roberto Cassar </p>

<div style="background-color: lightblue; font-size: 18px; padding: 10px;">
<div style="text-align: justify"><strong>Objetivo:</strong> Implementação do Algorítmo Genético </div>

## 🪨 **Introdução**

<p align="justify">
A dureza, é um valor quantitativo de resistência a deformações em um material. A partir desse conceito, o minerologista alemão Friedrich Mohs criou a escala de Mohs, que é uma escala da dureza de minerais com o objetivo de identifica-los em campo, testando os minerais contra alguns objetos, como a unha, uma moeda ou um prego. Cada resistência do material ao objeto representa um valor dentro da escala de Mohs, contendo valores de 1 a 10, quanto maior, maior a dureza do mineral.
</p>

<p align="justify">
O dataset usado no trabalho foi retirado dos estudos esperimentais reportados no "Physical and Optical Properties of Minerals CRC Handbook of Chemistry and Physics" e "The American Mineralogist Crystal Structure Database." Os atributos presentes no dataset são: Número de eletrons, Número de eletrons na camada de valência, Eletronegatividade de Pauling no estado de oxidação mais comum, Raios atomicos covalentes, Raio de Van der Walls e energia de ionização do neutro
</p>

<p align="justify">
Nesse trabalho, queremos encontrar o candidato que possua as propriedades que maximizem a dureza, visto que o material com maior dureza são mais resistentes ao desgaste e mais duráveis. Assim, entramos em um problema de <b>otimização de maximização</b>, que consiste em encontrar um ponto ótimo que maximiza a função objetivo. No futuro, será possivel encontrar a fórmula química do material sintetizado a partir das propriedades otimizadas pelo nosso algoritmo genético, visando a maximização da dureza.
</p>

<p align="justify">

</p>

<p align="justify">
Para a encontrar o candidato que buscamos, iremos usar os Algoritmos Genéticos para identifica-los, com uma função objetivo que possua um modelo de Aprendizado de Maquina, inicialmente testada com k-NN e Árvore de Decisão, para prever a dureza e, por fim, encontrar-mos o minério com a maior dureza.
</p>

## 📚**Importação de bibliotecas & Dataset**

Em primeiro lugar, precisamos importar as bibliotecas necessárias para a resolução do problema. Dentre essas, destaca-se o Pandas, para a importação e visualização do dataframe e Pickle, para a importação do dicionário que foi utilizado como base para a definição dos intervalos de cada *feature*.

In [1]:
import pandas as pd

import pickle
import pprint

from funcoes_tiamat import cria_populacao_feature as cria_populacao
from funcoes_tiamat import funcao_objetivo_pop_feature as funcao_objetivo_ag
from funcoes_tiamat import selecao_elitismo_torneio as funcao_selecao
from funcoes_tiamat import cruzamento_ponto_simples as funcao_cruzamento
from funcoes_tiamat import mutacao_simples as funcao_mutacao

C:\Users\julia24002\AppData\Roaming\Python\Python311\site-packages\pandas\core\arrays\masked.py:60: UserWarning: Pandas requires version '1.3.6' or newer of 'bottleneck' (version '1.3.5' currently installed).
  from pandas.core import (


Aqui, podemos visualizar o dataset que foi utilizado como base para a implementação do do algoritmo genético. De forma mais completa, é possível conferir o tratamento e utilização desse dataset no notebook "5.2 - Treinamento do modelo & Função objetivo".

In [2]:
df = pd.read_csv("Mineral_Dataset_Supplementary_Info.csv").dropna().drop(columns=["Unnamed: 0"])
display(df)

,Hardness,allelectrons_Total,density_Total,allelectrons_Average,val_e_Average,atomicweight_Average,ionenergy_Average,el_neg_chi_Average,R_vdw_element_Average,R_cov_element_Average,zaratio_Average,density_Average
0,2.3,110.0,23.000000,36.666667,2.666667,82.598467,8.504133,2.146667,2.006667,1.253333,0.456803,7.666667
1,5.5,406.0,30.472136,9.902439,4.682927,19.813180,11.456151,2.700244,1.676829,0.868293,0.522909,0.743223
2,5.5,406.0,30.472464,10.410256,4.923077,20.931371,11.541405,2.753590,1.703846,0.894359,0.497498,0.781345
3,5.5,476.0,61.142136,11.609756,4.682927,23.659644,11.487395,2.763659,1.714634,0.848780,0.519474,1.491272
4,5.5,476.0,61.142464,12.205128,4.923077,24.975089,11.574251,2.820256,1.743590,0.873846,0.493887,1.567755
...,...,...,...,...,...,...,...,...,...,...,...,...
617,3.8,46.0,9.133000,23.000000,4.000000,48.719500,9.877100,2.115000,1.905000,1.120000,0.478880,4.566500
618,4.5,86.0,6.674328,14.333333,5.166667,30.645954,11.862733,2.861667,1.700000,0.901667,0.487172,1.112388
619,4.0,38.0,7.134332,19.000000,4.000000,40.689515,11.506150,2.545000,1.765000,0.920000,0.479405,3.567166
620,7.5,86.0,8.841328,14.333333,5.000000,30.550687,11.543000,2.831667,1.735000,0.890000,0.489507,1.473555


Em relação as colunas do dataset, temos que:

| Features    | Descrição                                                                                      |
|-----------|-----------------------------------------------------------------------------------------------|
| `Hardness`   | Dureza                                                |
| `allelectrons_Total`  | Números de eletrons totais                                                     |
| `density_Total`  | Densidade total               |
| `allelectrons_Average`  | Média do número de eletrons              |
| `val_e_Average`  | Eletrons na camada de valência              |
| `atomicweight_Average`  | Peso atômico médio               |
| `ionenergy_Average`  | Eletronegatividade de Pauling no estado de oxidação mais comum              |
| `el_neg_chi_Average`  | Média da quantidade de pontos de concavidade nos contornos               |
| `R_vdw_element_Average`  | Raio de Van der Walls               |
| `R_cov_element_Average`  | Raios atomicos covalentes               |
| `zaratio_Average`  | Raio Atômico médio               |
| `density_Average`  | Densidade média               |


## 🧬**Rodando nosso Algorítmo Genético**

<p align="justify">
Para o funcionamento do nosso Algorítmo Genético, inicialmente criamos uma população, a partir do módulo random.int, respeitando os valores mínimos e máximos presentes em cada feature ("5.2 - Análise dos dados & Cria candidato"). Logo em seguida, calculamos o valor de fitness a partir da função objetivo ("5.2 Treinamento do modelo - Função Objetivo"), o qual utiliza um modelo de machine learning k-NN para prever novos valores de dureza para cada indivíduo da população. Como operadores, usamos dois operadores de seleção combinados: Elistismo e Torneio, com o objetivo de selecionar os 5 indivíduos com os maiores valores de fitness por elitismo e depois selecionar os demais maiores indivíduos a partir do torneio.  
</p>

<p align="justify">
Além disso, o operador de cruzamento utilizado foi o de ponto simples, a fim de misturar genes de indivíduos pré-existentes, gerando novos. Por fim, a operação de mutação, também utilizando o módulo random.int, foi criada para trocar o gene sorteado por um outro valor, ainda respeitando os máximos e mínimos de cada *feature*. Assim, depois de todos os operadores, a função objetivo prevê os valores de dureza de todos esses indivíduos e atualiza a população.
</p>

Inicialmente, buscamos o dicionário que está armazenado em um arquivo produzido pela biblioteca pickle. E criamos uma variável com todos os intervalos de mínimo e máximo presentes no dicionário. Essa será utilizada nas funções de criação de população e mutação.

In [3]:
with open("dicionario_intervalos.pkl", "rb") as dicionario_range:
    dicionario_range = pickle.load(dicionario_range)

Vale ressaltar que a *feature* não será utilizada na função de mutação e, portanto, pode ser retirada dessa variável.

In [4]:
dicionario_range.pop("Hardness")


[1, 10.0]

In [5]:
INTERVALOS = [i for i in dicionario_range.values()]

Agora, nosso algorítmo genético irá rodar 50 gerações, ou seja, 50 vezes por todas essas etapas. Porém, houve a implementação de uma estrátegia análoga ao *Early Stopping* utilizado em redes neurais, para caso o melhor indivíduo predito fique sendo escolhido repetidas vezes por um determinado número de gerações - nesse caso, 10.

Para as variáveis utilizadas para a construção do algoritmo genético:
* **Tamanho da população**: Considerando a complexidade do espaço de busca, cada geração será composta por 500 indivíduos;

* **Taxa de cruzamento**: Fixada em 0.5 (ou 50%), define a probabilidade de dois indivíduos — "pai" e "mãe" — serem cruzados para a geração de dois novos indivíduos;

* **Taxa de mutação**: Estabelecida em 10%

* **Tamanho do torneio**: Cada seleção por torneio considerará 5 indivíduos, dentre os quais o melhor será escolhido; Vale ressaltar que o torneio, nesse contexto, será utilizado juntamente com o operador genético de elitismo, o qual selecionará, inicialmente, os 5 melhores indivíduos da geração anterior. * 

* **Número máximo de gerações**: O algoritmo terá até 100 gerações para tentar encontrar encontrar um indivíduo que apresente uma dureza acima de 8. Caso 10 gerações se passem com o algoritmo prevendo o mesmo valor, o algoritmo irá parar de rodar.

In [6]:
TAMANHO_POPULACAO = 10000
NUM_GERACOES = 50
CHANCE_DE_CRUZAMENTO = 0.5
CHANCE_DE_MUTACAO = 0.1
TAMANHO_TORNEIO = 5

Antes de rodar o algoritmo, precisamos apensa criar a população.

In [7]:
populacao = cria_populacao(TAMANHO_POPULACAO, INTERVALOS)
#pprint(populacao)
print(populacao)

[[5719, 335, 60, 8, 153, 10, 4, 4, 2, 2, 3], [12534, 306, 17, 6, 93, 8, 2, 1, 3, 2, 10], [13210, 621, 23, 9, 144, 9, 3, 4, 3, 2, 1], [3989, 118, 26, 4, 94, 11, 2, 2, 2, 4, 11], [813, 492, 61, 2, 53, 8, 4, 1, 4, 1, 10], [5929, 419, 36, 4, 106, 9, 3, 3, 1, 0, 13], [5591, 284, 28, 7, 55, 11, 4, 1, 1, 0, 1], [9728, 324, 63, 3, 136, 8, 4, 1, 1, 0, 8], [14530, 392, 28, 7, 138, 8, 4, 4, 2, 3, 4], [13526, 422, 40, 3, 19, 12, 2, 4, 3, 2, 4], [12972, 547, 42, 9, 158, 13, 3, 1, 3, 4, 4], [5042, 196, 60, 7, 11, 9, 2, 4, 1, 4, 4], [1460, 626, 58, 6, 92, 11, 3, 2, 1, 0, 5], [14038, 414, 49, 4, 55, 13, 2, 2, 1, 3, 14], [2482, 437, 47, 9, 62, 13, 2, 4, 2, 0, 11], [12392, 232, 21, 2, 53, 10, 2, 1, 2, 2, 8], [9670, 362, 17, 5, 106, 12, 2, 3, 4, 4, 9], [11467, 213, 54, 7, 15, 14, 2, 4, 3, 1, 11], [8609, 527, 60, 4, 51, 10, 2, 2, 4, 2, 2], [14761, 602, 61, 2, 100, 9, 3, 3, 1, 2, 3], [7969, 123, 39, 5, 62, 9, 3, 1, 4, 1, 1], [8608, 83, 9, 8, 39, 9, 3, 4, 3, 4, 7], [14767, 66, 47, 9, 109, 10, 2, 3, 1, 0, 10

In [8]:
hall_da_fama = []
maiores_fitness = []
paciencia = 0

contador = 0
maior_fitness = 0
 
while maior_fitness < 9 and paciencia <= 10 and contador < 100:
    contador += 1

    # Seleção
    fitness = funcao_objetivo_ag(populacao)        
    selecionados = funcao_selecao(populacao, fitness, TAMANHO_TORNEIO)

    # Cruzamento
    proxima_geracao = []
    for pai, mae in zip(selecionados[::2], selecionados[1::2]):
        individuo1, individuo2 = funcao_cruzamento(pai, mae, CHANCE_DE_CRUZAMENTO)
        proxima_geracao.append(individuo1)
        proxima_geracao.append(individuo2)

    # Mutação
    funcao_mutacao(proxima_geracao, CHANCE_DE_MUTACAO, INTERVALOS)

    # Nova avaliação
    fitness = funcao_objetivo_ag(proxima_geracao)
    maior_fitness = max(fitness)
    maiores_fitness.append(maior_fitness)

    if len(maiores_fitness) > 1:
        diferenca = abs(maior_fitness - maiores_fitness[-2])
    else:
        diferenca = float('inf')  

    if diferenca < 0.1:
        paciencia += 1
    else:
        paciencia = 0

    indice = fitness.index(maior_fitness)
    hall_da_fama.append(proxima_geracao[indice])    

    # Atualiza população
    populacao = proxima_geracao

    print(f"Geração {contador}: maior fitness = {maior_fitness}")


Geração 1: maior fitness = [6.]
Geração 2: maior fitness = [6.]
Geração 3: maior fitness = [6.26]
Geração 4: maior fitness = [6.26]
Geração 5: maior fitness = [7.1]
Geração 6: maior fitness = [7.36]
Geração 7: maior fitness = [7.66]
Geração 8: maior fitness = [7.46]
Geração 9: maior fitness = [7.82]
Geração 10: maior fitness = [7.82]
Geração 11: maior fitness = [7.82]
Geração 12: maior fitness = [7.82]
Geração 13: maior fitness = [7.82]
Geração 14: maior fitness = [7.82]
Geração 15: maior fitness = [7.82]
Geração 16: maior fitness = [7.82]
Geração 17: maior fitness = [7.82]
Geração 18: maior fitness = [7.82]
Geração 19: maior fitness = [7.82]
Geração 20: maior fitness = [7.82]


## 📊 **Apresentando o melhor indivíduo com maior dureza**

Para isso, observamos o primeiro elemento do hall da fama, onde se encontra o indivíduo mais otimizado.

In [10]:
melhor_individuo = hall_da_fama[0]
print(melhor_individuo)

[6684, 53, 7, 3, 139, 13, 4, 1.7937298992456312, 2, 0, 8.638035496554332]


Apenas para garantir que que esses estajam dentro do intervalo definido, podemos observar novamente os intervalos definidos.

In [9]:
dicionario_range

{'allelectrons_Total': [6, 15300.0],
 'density_Total': [1, 645.0],
 'allelectrons_Average': [4, 70.0],
 'val_e_Average': [2, 10.0],
 'atomicweight_Average': [8, 170.0],
 'ionenergy_Average': [8, 15.0],
 'el_neg_chi_Average': [2, 5.0],
 'R_vdw_element_Average': [1, 5.0],
 'R_cov_element_Average': [1, 5.0],
 'zaratio_Average': [0, 5.0],
 'density_Average': [0, 15.0]}

Parece tudo dentro da conformidade!

* Vale ressaltar que o valor 0 é explicado pelo próprio intervalo: a *feature*: zratio possui um intervalo menor do que um (apenas temos que o intervalo foi extendido até 5 pela regra utilizada para a criação dos intervalos).

Podemos comparar as *features* do indivíduo obtido com aquelas dos indivíduos do dataset original que estão no mesmo intervalo de dureza.

In [20]:
df_filtrado_natural = (df['Hardness'] >= 7.5) & (df['Hardness'] <= 8)
display(df[df_filtrado_natural])

,Hardness,allelectrons_Total,density_Total,allelectrons_Average,val_e_Average,atomicweight_Average,ionenergy_Average,el_neg_chi_Average,R_vdw_element_Average,R_cov_element_Average,zaratio_Average,density_Average
45,7.5,80.0,7.734660,10.000000,5.000000,20.255278,11.026725,2.790000,1.672500,0.852500,0.495275,0.966832
87,7.8,266.0,24.945976,9.172414,4.965517,18.534006,11.516403,2.801724,1.663103,0.821034,0.492634,0.860206
200,7.5,72.0,6.883744,8.000000,4.444444,16.120079,11.683233,2.720000,1.574444,0.765556,0.546261,0.764860
258,7.8,88.0,12.536328,12.571429,4.571429,26.191314,10.834029,2.661429,1.681429,0.891429,0.488927,1.790904
259,7.8,83.0,12.843328,11.857143,4.571429,24.699606,10.554000,2.647143,1.687143,0.904286,0.488394,1.834761
290,7.5,46.0,6.071412,5.750000,4.000000,11.729291,11.876775,2.642500,1.520000,0.712500,0.542791,0.758926
291,7.5,46.0,6.071576,6.571429,4.571429,13.689408,12.174357,2.782857,1.572857,0.762857,0.474844,0.867368
303,7.8,84.0,13.277328,12.000000,4.571429,24.829171,10.620914,2.687143,1.685714,0.897143,0.489894,1.896761
321,7.8,109.0,23.193328,15.571429,4.285714,32.946309,11.101600,2.710000,1.744286,0.904286,0.483751,3.313333
348,7.8,76.0,15.751996,15.200000,4.400000,31.937418,11.331820,2.796000,1.728000,0.880000,0.486236,3.150399


In [22]:
melhor_individuo = hall_da_fama[0]
print(melhor_individuo)

[6684, 53, 7, 3, 139, 13, 4, 1.7937298992456312, 2, 0, 8.638035496554332]


A fim de comparar os atributos, podemos comparar os valores em relação a cada *feature*. Ao lado da coluna, dentro de um colchete, serão apresentados, respectivamente, os valores previstos e reais:
* "allelectrons_Total" [6684, 419]: Apresenta valor bem distante de todas as instâncias, mas apresenta maior similaridade com a instância 526 (maior valor);
* "density_Total" [53, 49.39]: Também apresenta maior similaridade com a instância 526, com valor próximo a 50;
* "allelectrons_Average" [7, 8]: A instância 200 apresenta valor mais próximo;
* "val_e_Average" [3, 4]: Todos os materiais listados apresentam um valor maior para esse atributo e, portanto, o menor valor para esse atributo, presente na instância 290 é o mais próximo;
* "atomicweight_Average" [139, 34.48]: A previsão apresenta valor bem distante dos valores reais, sendo o mais próximo aquele com maior valor, correspondente a instância 576; 
* "ionenergy_Average" [13, 12.16]: Os valores são mais próximos, considerando o intervalo da *feature*, mas o material apresenta valor maior do que os demais, estando mais próximo, portanto, do de maior valor, presente na instância 526;
* "el_neg_chi_Average" [4, 2.98]: Assim, como no anterior, os valores são próximos, mas ainda assim a previsão apresenta valor maior do que todas as instâncias listadas. Nesse sentido, a instância mais próxima é a 567;
* "R_vdw_element_Average" [1.79, 1.75]: Os valores também estão acima do real, mas próximos, considerando o range da feature, sendo a instância mais próxima a 321;
* "R_cov_element_Average" [2, 0.92]: A instância mais próxima foi a 523, que apresentou o maior valor;
* "zaratio_Average" [0, 0.47]: A instância que apresentou valor mais próximo de 0 foi a 291;
* "density_Average" [8.63, 3.52]: O maior valor foi o da instância 567, sendo mais próxima ao valor previsto.

Por fim, comparamos com o indivíduo com uma dureza parecida do dataset de cristais artificiais que contm as mesmas features que as dos minerais. Aqui, iremos utilizar apenas os materiais com dureza correspondente a 8 (nosso objetivo central).

In [11]:
print(melhor_individuo)

[6684, 53, 7, 3, 139, 13, 4, 1.7937298992456312, 2, 0, 8.638035496554332]


In [ ]:
df_artificial = pd.read_csv("Artificial_Crystals_Dataset.csv").
df_filtrado = df_artificial[df_artificial["Hardness (Mohs)"] == 8]

df_filtrado

,Formula,Crystal structure,Hardness (Mohs),allelectrons_Total,density_Total,allelectrons_Average,val_e_Average,atomicweight_Average,ionenergy_Average,el_neg_chi_Average,R_vdw_element_Average,R_cov_element_Average,zaratio_Average,density_Average
31,MgAl2O4,cubic,8.0,70.0,7.143328,10.00000,4.571429,20.323314,10.584314,2.612857,1.641429,0.920000,0.493919,1.020475
42,Nd0.02YAl3(BO3)4,monoclinic,8.0,195.2,22.199984,9.75025,4.747253,20.377006,11.033686,2.772867,1.688701,0.827053,0.486636,1.108890


O candidato encontrado tem maior similaridade com a instância 42, sendo que algumas *features*, apresentaram valores semelhantes, com "val_e_Average", "el_neg_chi_Average" e "R_cov_element_Average". No entanto, assim como ocorrido no dataset original, algumas apresentam valores bem distantes como "allelectrons_Total", "atomicweight_Average" e "density_Average".

## 😁 **Conclusão**

<p align="justify">
Após a utilização de um algoritmo genético para a descoberta de materiais com maior dureza, conseguimos obter um candidato com dureza próxima ao intervalo mínimo inicialmente desejado (8). Para essa composição, utilizamos operadores como de cruzamento de ponto duplo, seleção combinada de torneio e elitismo e mutação simples. Como principais modificações, criamos uma função objetivo feita a partir de um modelo de aprendizado de máquina (k-NN), para a previsão da dureza dos candidatos, bem como uma função de cria população que obedece o intervalo original das <em>features</em>.
<p>

<p align="justify">
Ao longo da análise do candidato obtido, identificamos a presença de valores extremos ("allelectrons_Total", "atomicweight_Average" e "density_Average"), o que sugere a necessidade de aplicar restrições adicionais, em perspectivas futuras, para uma compreensão mais precisa da relação dessas features com outras variáveis. 
</p>

<p align="justify">
Ao final, foi possível observar que o candidato também possui algumas características que são parecidas com o cristais artificiais e reais de dureza parecida, como o "val_e_Average", 'el_neg_chi_Average' e "R_cov_element_Average". Dessa forma, foi possível observar um indivíduo com características novas, em relação aos apresentados na literatura.
</p>